In [21]:
#Step 1 install dependencies and load our fine-tuned model
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.1.6
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')
tokenizer = AutoTokenizer.from_pretrained("huggingtweets/elonmusk")  
model = AutoModelWithLMHead.from_pretrained("huggingtweets/elonmusk")
model.eval()
model.cuda()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-zuoaggvk/pfrl_a82459c286ad4bd68eda8ae069510536
  Running command git clone --filter=blob:none --quiet https://github.com/voidful/pfrl.git /tmp/pip-install-zuoaggvk/pfrl_a82459c286ad4bd68eda8ae069510536
  Resolved https://github.com/voidful/pfrl.git to commit 2ad3d51a7a971f3fe7f2711f024be11642990d61
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [33]:
# - Define a Reward model
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)
sentiment("dogecoin is bad")
sentiment("dogecoin is bad")[0][0]['score']
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        if 1 < len(predicted_list):
          predicted_text = tokenizer.convert_tokens_to_string(predicted_list)
          # sentiment classifier
          reward += sentiment(input_item[0]+predicted_text)[0][2]['score']
      return reward

In [31]:
sentiment("dogecoin is ok")

[[{'label': 'LABEL_0', 'score': 0.01306374091655016},
  {'label': 'LABEL_1', 'score': 0.3054283857345581},
  {'label': 'LABEL_2', 'score': 0.6815078854560852}]]

In [34]:
observaton_list = [['tesla is']]

#Create a Reinforcement Learning Environment i.e Markov Decision Process

# initialize the reward model using our fine-tuned model and one example prompt 
env = MyRLEnv(model, tokenizer, observation_input=observaton_list)

# intialize the agent using the pre-trained model
actor = TextRLActor(env,model,tokenizer)

# Run reinforcement learning via PPO technique to maximize reward
agent = actor.agent_ppo(update_interval=10, minibatch_size=10, epochs=10)

# Test it
actor.predict(observaton_list[0])

' a real problem.'

In [35]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=100,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='tesla', 
)

(<textrl.actor.TextPPO at 0x7f136b6f3070>,
 [{'average_value': 2.2727895,
   'average_entropy': 0.21261017,
   'average_value_loss': 2.8386925935745237,
   'average_policy_loss': -0.0921926562488082,
   'n_updates': 10,
   'explained_variance': nan,
   'eval_score': 0.9697455167770386},
  {'average_value': 1.4515439,
   'average_entropy': 0.21450341,
   'average_value_loss': 2.018683874607086,
   'average_policy_loss': -0.051273139180848526,
   'n_updates': 20,
   'explained_variance': -3.0870882268405664,
   'eval_score': 0.9875895380973816},
  {'average_value': 1.2768729,
   'average_entropy': 0.18776588,
   'average_value_loss': 1.59937850634257,
   'average_policy_loss': -0.05631556159894302,
   'n_updates': 30,
   'explained_variance': -36.592487966567504,
   'eval_score': 0.005883386824280024},
  {'average_value': 1.2428812,
   'average_entropy': 0.19033684,
   'average_value_loss': 1.2586446049809457,
   'average_policy_loss': -0.04666200192878026,
   'n_updates': 50,
   'explai

In [36]:
# - Evaluate
agent.load("./tesla/best")
actor.predict(observaton_list[0])

' the most beautiful part of the world'